In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import scripts
import attack

import matplotlib.pyplot as plt
import json 

import os

import random



In [32]:
# purchase
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))
X_train_size = x_target_train.shape[0]
X_test_size = x_target_test.shape[0]


In [ ]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

In [4]:
#Load model and parameters
file = 'tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model_params.json'
with open(file) as json_file:
    tm_params = json.load(json_file)
            
# x_target_train, y_target_train, x_target_test, y_target_test = attack.data_shuffle(rand_seed, X_raw, y_raw)
target_model = algo.LogisticRegression_DPSGD()
scripts.set_model_params(target_model, tm_params)
target_model.theta = np.load('tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model.npy')
target_model.evaluate(x_target_train, y_target_train, acc=True)
target_model.evaluate(x_target_test, y_target_test, acc=True)

#Output DP
noise_model = scripts.output_DP(target_model, epsilon_out_DP=10, delta_out_DP=1e-5, X_train_size=X_train_size)
noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=True)
    
    

The accuracy of the model : 82.69999999999999 %
The accuracy of the model : 52.6 %
The accuracy of the model : 77.5 %
The accuracy of the model : 48.6 %


# FL output DP

In [5]:
path = 'fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            for k in results[r]['Unnamed: 0']:
                it.append(k[k.find('i')+1:k.find('_')])
                client.append(k[k.find('_')+1:])
            results[r]['it'] = it
            results[r]['client'] = client
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [6]:
fl_file = 'fl/rs42_ncl2_fiter5_lr0.001_iter100_reg1e-05_DPFalse'
# results[fl_file]

In [69]:
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

epsilon_out_dp = 10

number_of_clients = len(results[fl_file]['client'].unique())-1
data_per_client = int(x_target_train.shape[0]/number_of_clients)
clients = results[fl_file]['client'].unique()
fl0 = {}
fl0_out_dp = {}

for i,c in enumerate(clients):
    fl0[c] = algo.LogisticRegression_DPSGD()
    scripts.set_model_params(fl0[c], params[fl_file])
    if c == 'g':
        fl0[c].x = x_target_train
        fl0[c].y = y_target_train
        fl0[c].theta = models[fl_file][f'i0_{c}.npy']
        fl0_out_dp[c] = algo.LogisticRegression_DPSGD()
        scripts.set_model_params(fl0_out_dp[c], fl0_out_dp['c0'].__dict__)
        fl0_out_dp[c].theta = sum([fl0_out_dp[c].theta for c in fl0_out_dp if c!='g'])/number_of_clients
    else:
        fl0[c].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
        fl0[c].y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        fl0[c].theta = models[fl_file][f'i0_{c}.npy']
        fl0_out_dp[c] = scripts.output_DP(target_model, fl0[c].x.shape[0], epsilon_out_dp)
    fl0_out_dp[c].train_acc = fl0[c].evaluate(fl0[c].x, fl0[c].y)
    fl0_out_dp[c].test_acc = fl0[c].evaluate(x_target_test, y_target_test)
    fl0_out_dp[c].out_dp_train_acc = fl0_out_dp[c].evaluate(fl0[c].x, fl0[c].y)
    fl0_out_dp[c].out_dp_test_acc = fl0_out_dp[c].evaluate(x_target_test, y_target_test)
    fl0_out_dp[c].mi_attack = attack.test_mi_attack(ams, fl0[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)
    fl0_out_dp[c].out_dp_mi_attack = attack.test_mi_attack(ams, fl0_out_dp[c], fl0[c].x, fl0[c].y, x_target_test, y_target_test)

In [70]:
for c in fl0_out_dp:
    pars = fl0_out_dp[c].__dict__
    print(pars['mi_attack']['attack_acc_mean'])
    print(pars['out_dp_mi_attack']['attack_acc_mean'])
#     print(pars['theta'][0][0])

0.7522222222222221
0.6544888888888889
0.7507666666666667
0.6601
0.5042333333333334
0.6302333333333333


In [68]:
for c in fl0_out_dp:
    pars = fl0_out_dp[c].__dict__
    print(pars['mi_attack']['attack_acc_mean'])
    print(pars['out_dp_mi_attack']['attack_acc_mean'])
#     print(pars['theta'][0][0])

0.7522222222222221
0.5787444444444444
0.7507666666666667
0.5489666666666667
0.5042333333333334
0.5033416666666667


In [53]:
fl0_out_dp[c].__dict__['']

SyntaxError: invalid syntax. Perhaps you forgot a comma? (925288418.py, line 1)

In [78]:
a = fl0['c0'].__dict__['theta'] + fl0['c1'].__dict__['theta']
a/2 == fl0['g'].__dict__['theta']

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [79]:
a/2 

array([[1.05996405, 0.86456914, 1.0623417 , ..., 1.02371914, 1.11752676,
        1.08531036],
       [0.99950012, 0.99950012, 0.99950012, ..., 0.99950012, 0.99950012,
        0.99950012],
       [1.01371005, 1.01104277, 0.98958465, ..., 0.9959107 , 1.03070685,
        1.0596255 ],
       ...,
       [0.98830986, 1.04842534, 0.99321391, ..., 0.98871956, 1.01987505,
        1.02216046],
       [0.99243359, 1.05779925, 0.99031277, ..., 1.01283557, 1.06391893,
        1.00793724],
       [1.06505776, 1.05122922, 0.99219825, ..., 1.01403267, 0.97548126,
        0.98986542]])

In [80]:
fl0['g'].__dict__['theta']

array([[0.7066427 , 0.57637943, 0.7082278 , ..., 0.68247942, 0.74501784,
        0.72354024],
       [0.66633342, 0.66633342, 0.66633342, ..., 0.66633342, 0.66633342,
        0.66633342],
       [0.6758067 , 0.67402851, 0.6597231 , ..., 0.66394047, 0.6871379 ,
        0.706417  ],
       ...,
       [0.65887324, 0.69895022, 0.66214261, ..., 0.65914637, 0.6799167 ,
        0.68144031],
       [0.6616224 , 0.7051995 , 0.66020851, ..., 0.67522372, 0.70927929,
        0.67195816],
       [0.71003851, 0.70081948, 0.6614655 , ..., 0.67602178, 0.65032084,
        0.65991028]])